# 7.- Azure ML Compute

In [ ]:
import yaml

from azure.identity import DefaultAzureCredential

from azure.ai.ml import MLClient

from azure.ai.ml.entities import AmlCompute

## Define Variables

In [ ]:
# Load configuration from the YAML file
with open("../config.yaml", "r") as file:
    config = yaml.safe_load(file)

In [ ]:
subscription_id = config["azure"]["subscription_id"]
resource_group_name = config["azure"]["resource_group_name"]
workspace_name = config["azure"]["workspace_name"]

training_gpu_cluster = config["azure"]["training_gpu_cluster"]
compute_name = config["azure"]["compute_name"]

## Azure Authentication

In [ ]:
# Initialize DefaultAzureCredential
credential = DefaultAzureCredential()

In [ ]:
# Update Azure ML Client
ml_client = MLClient(credential, subscription_id, resource_group_name, workspace_name)

Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented


## Create a Compute Resource

In [ ]:
def create_compute_resource(ml_client, cluster_name, size="STANDARD_D2_V3", 
                            min_instances=0, max_instances=4, 
                            idle_time_before_scale_down=180, tier="Dedicated"):
    """
    Creates or reuses an AMLCompute resource in Azure ML.

    Parameters:
        ml_client (MLClient): The Azure ML client.
        cluster_name (str): The name of the compute cluster.
        size (str): The VM size (default is "STANDARD_D2_V3").
        min_instances (int): The minimum number of instances (default is 0).
        max_instances (int): The maximum number of instances (default is 4).
        idle_time_before_scale_down (int): Idle time in seconds before scaling down (default is 180).
        tier (str): The pricing tier, either "Dedicated" or "LowPriority" (default is "Dedicated").

    Returns:
        An instance of AmlCompute representing the compute resource.
    """
    try:
        # Check if the compute cluster already exists
        compute_resource = ml_client.compute.get(cluster_name)
        print(f"A cluster named '{cluster_name}' already exists; reusing it.")
    except Exception:
        print("Creating a new GPU compute resource...")
        compute_resource = AmlCompute(
            name=cluster_name,
            type="amlcompute",
            size=size,
            min_instances=min_instances,
            max_instances=max_instances,
            idle_time_before_scale_down=idle_time_before_scale_down,
            tier=tier,
        )
        # Create the compute resource and wait until the operation completes
        compute_resource = ml_client.begin_create_or_update(compute_resource).result()
    
    print(f"AMLCompute resource '{compute_resource.name}' is created with size '{compute_resource.size}'.")
    return compute_resource


In [ ]:
create_compute_resource(ml_client, 
                        training_gpu_cluster,
                        compute_name)

Creating a new GPU compute resource...
AMLCompute resource 'training-gpu-cluster' is created with size 'STANDARD_D2_V3'.


AmlCompute({'type': 'amlcompute', 'created_on': None, 'provisioning_state': 'Succeeded', 'provisioning_errors': None, 'name': 'training-gpu-cluster', 'description': None, 'tags': None, 'properties': {}, 'print_as_yaml': False, 'id': '/subscriptions/7a95526c-4313-494a-9c1d-e75dc44978b5/resourceGroups/test_group/providers/Microsoft.MachineLearningServices/workspaces/machine_que_tal/computes/training-gpu-cluster', 'Resource__source_path': '', 'base_path': 'c:\\Users\\walte\\Documents\\Projects\\GitHub\\My_Organizations\\Computer-Vision-WSC\\yolo11-segmentation\\notebooks', 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x000001A6EA7DA170>, 'resource_id': None, 'location': 'eastus', 'size': 'STANDARD_D2_V3', 'min_instances': 0, 'max_instances': 4, 'idle_time_before_scale_down': 180.0, 'identity': None, 'ssh_public_access_enabled': True, 'ssh_settings': None, 'network_settings': <azure.ai.ml.entities._compute.compute.NetworkSettings object at 0x000001A6EA7